<a href="https://colab.research.google.com/github/hahaeaezz-arch/chaemin-lee/blob/main/Restaurant_Monthly_Revenue_Prediction_(Regression).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from tensorflow import keras
from keras import Sequential, Input
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [17]:
df = pd.read_csv("/content/drive/MyDrive/딥러닝_과제 모음/식당 월 매출 예측/train.csv")
df.head(3)

,Number_of_Customers,Menu_Price,Marketing_Spend,Cuisine_Type,Average_Customer_Spending,Promotions,Reviews,Monthly_Revenue
0,57,43.513029,8.821443,Japanese,44.117055,1,90,249.135286
1,20,43.661186,17.054560,Italian,39.314367,0,65,202.232364
2,17,27.977353,6.795377,Mexican,43.798834,1,88,231.041234


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Number_of_Customers        800 non-null    int64  
 1   Menu_Price                 800 non-null    float64
 2   Marketing_Spend            800 non-null    float64
 3   Cuisine_Type               800 non-null    object 
 4   Average_Customer_Spending  800 non-null    float64
 5   Promotions                 800 non-null    int64  
 6   Reviews                    800 non-null    int64  
 7   Monthly_Revenue            800 non-null    float64
dtypes: float64(4), int64(3), object(1)
memory usage: 50.1+ KB


In [19]:
df["Cuisine_Type"].value_counts()

,count
Cuisine_Type,
Japanese,210
American,205
Mexican,200
Italian,185


In [20]:
df = pd.get_dummies(df)
df.head(3)

,Number_of_Customers,Menu_Price,Marketing_Spend,Average_Customer_Spending,Promotions,Reviews,Monthly_Revenue,Cuisine_Type_American,Cuisine_Type_Italian,Cuisine_Type_Japanese,Cuisine_Type_Mexican
0,57,43.513029,8.821443,44.117055,1,90,249.135286,False,False,True,False
1,20,43.661186,17.054560,39.314367,0,65,202.232364,False,True,False,False
2,17,27.977353,6.795377,43.798834,1,88,231.041234,False,False,False,True


In [21]:
df.columns

Index(['Number_of_Customers', 'Menu_Price', 'Marketing_Spend',
       'Average_Customer_Spending', 'Promotions', 'Reviews', 'Monthly_Revenue',
       'Cuisine_Type_American', 'Cuisine_Type_Italian',
       'Cuisine_Type_Japanese', 'Cuisine_Type_Mexican'],
      dtype='object')

In [25]:
x_train = df[['Number_of_Customers', 'Menu_Price', 'Marketing_Spend',
       'Average_Customer_Spending', 'Promotions', 'Reviews', 'Monthly_Revenue',
       'Cuisine_Type_American', 'Cuisine_Type_Italian',
       'Cuisine_Type_Japanese', 'Cuisine_Type_Mexican']]
y_train = df['Monthly_Revenue']

In [32]:
model = Sequential()
model.add(Input(shape=(x_train.shape[1],)))
model.add(Dense(units = 30, activation = 'relu'))
model.add(Dense(units = 10, activation = 'relu'))
model.add(Dense(units = 1 , activation = 'relu'))

model.compile(loss="mean_squared_error", optimizer="adam")

mc = ModelCheckpoint("best-model.keras", save_best_only=True)
es = EarlyStopping(patience=2, restore_best_weights=True)

history = model.fit(x_train, y_train, validation_split=0.25, epochs=100, callbacks=[mc, es])

print("/content/drive/MyDrive/딥러닝_과제 모음/식당 월 매출 예측/train.csv:", model.evaluate(x_train, y_train, verbose=0))

Epoch 1/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 52063.6367 - val_loss: 35636.2812
Epoch 2/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 29026.7520 - val_loss: 17334.0371
Epoch 3/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 11964.4736 - val_loss: 5734.4702
Epoch 4/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 3619.0852 - val_loss: 1355.6512
Epoch 5/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 1221.0448 - val_loss: 1006.3798
Epoch 6/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1020.8444 - val_loss: 823.0845
Epoch 7/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 824.0317 - val_loss: 691.3021
Epoch 8/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 651.5530 - val_loss: 513.0306
Epoch 9/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 465.2179 - val_loss: 315.4541
Epoch 10/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 272.5778 - val_loss: 200.5949
Epoch 11/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 201.2903 - val_loss: 1

In [38]:
import pandas as pd
import numpy as np

sub = pd.read_csv("/content/drive/MyDrive/딥러닝_과제 모음/식당 월 매출 예측/submission.csv")
sub.dropna(axis=1, inplace=True)

test_df = pd.read_csv("/content/drive/MyDrive/딥러닝_과제 모음/식당 월 매출 예측/test.csv")

train_cols = x_train.columns
x_test = pd.get_dummies(test_df).reindex(columns=train_cols, fill_value=0)

y_pred = model.predict(x_test, verbose=0).reshape(-1)

sub["Monthly_Revenue"] = y_pred
sub.to_csv("./new_submission.csv", index=False)
